# Interview task

## check GPU

In [27]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda:0


## Check dataset

In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('substance_interactions.csv')

In [30]:
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_START_INDEX,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1329,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,912,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1541,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,741,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,480,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n


In [31]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL'],
      dtype='object')

In [32]:
def pre_processing(example):
    sentence = example['SENTENCE']
    subject = example['SUBJECT_TEXT']
    object = example['OBJECT_TEXT']
    relation = example['PREDICATE']
    # text = f"{subject} [SEP] {relation} [SEP] {object} [SEP] {sentence}"
    text = f"{sentence} [SEP] {subject} , {relation} , {object}"
    return text

df['triple_with_sentence'] = df.apply(pre_processing,axis=1)
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL,triple_with_sentence
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n,"Nor did administration of SA, diflunisal or AS..."
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n,A comparative study of recombinant L-cha...
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y,These findings suggest that some nicotinic alk...
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y,With a truncated chimaeric IL-5Rbeta-gp1...
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n,"Neutral endopeptidase, a zinc-dependent ..."


## Tokenizer

In [33]:
from transformers import AutoTokenizer
distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [34]:
text = 'Hello everyone! ! antidisestablishmentarianism'
tokenizer = distilbert_tokenizer

print(f'\n{tokenizer.name_or_path}')
vocab = {v: k for k, v in tokenizer.vocab.items()}
tokenized_text = tokenizer(text)
print([vocab[id] for id in tokenized_text['input_ids']])


distilbert-base-uncased
['[CLS]', 'hello', 'everyone', '!', '!', 'anti', '##dis', '##est', '##ab', '##lish', '##ment', '##arian', '##ism', '[SEP]']


#### test tokenizer for a triple with corresponding sentence

In [35]:
example = df['triple_with_sentence'].iloc[0]
example

'Nor did administration of SA, diflunisal or ASA itself impair the       anti-aggregatory effect of a fresh test dose of ASA. [SEP] SA , INTERACTS_WITH , ASA'

In [36]:
tokenizer(example, return_tensors='pt', truncation=True)

{'input_ids': tensor([[  101,  4496,  2106,  3447,  1997,  7842,  1010,  4487, 10258, 19496,
         12002,  2030, 17306,  2993, 17727, 11215,  1996,  3424,  1011, 24089,
          2100,  3466,  1997,  1037,  4840,  3231, 13004,  1997, 17306,  1012,
           102,  7842,  1010, 11835,  2015,  1035,  2007,  1010, 17306,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [37]:
def processing(example):
    res = tokenizer(example['triple_with_sentence'])
    # res['label'] = example['LABEL']
    res['label'] = 1 if example['LABEL']=='y' else 0
    return res
df['data'] = df.apply(processing, axis=1)

In [38]:
df['data'][0]

{'input_ids': [101, 4496, 2106, 3447, 1997, 7842, 1010, 4487, 10258, 19496, 12002, 2030, 17306, 2993, 17727, 11215, 1996, 3424, 1011, 24089, 2100, 3466, 1997, 1037, 4840, 3231, 13004, 1997, 17306, 1012, 102, 7842, 1010, 11835, 2015, 1035, 2007, 1010, 17306, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 0}

In [39]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL', 'triple_with_sentence', 'data'],
      dtype='object')

## split the data, training set 70%, validation set 15%, test set 15%

In [40]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [41]:
print(len(train_data),len(val_data),len(test_data))

2100 450 450


In [42]:
train_data = train_data.reset_index()
val_data = val_data.reset_index()
test_data = test_data.reset_index()

In [43]:
from transformers import DataCollatorWithPadding
import evaluate

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load('accuracy')

Using the latest cached version of the module from C:\Users\91338\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Fri Feb  2 12:22:07 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [44]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)
def compute_metrics(eval_pred):
	    predictions, labels = eval_pred
	    predictions = np.argmax(predictions, axis=1)
	    
	    # Calculate precision, recall, and F1 score
	    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
	    
	    return {
	        'accuracy': accuracy_score(labels, predictions),
	        'precision': precision,
	        'recall': recall,
	        'f1': f1
	    }

## BERT model

In [45]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM

labels = ['n', 'y']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

print('id2label:', id2label)
print('label2id:', label2id)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id)

# Only train last classifier layer
# for param in model.base_model.parameters():
#     param.requires_grad = False

id2label: {0: 'n', 1: 'y'}
label2id: {'n': 0, 'y': 1}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [47]:
# Freeze all layers except the last one
for param in model.base_model.parameters():
    param.requires_grad = False

for param in model.pre_classifier.parameters():
    param.requires_grad = True

# # Unfreeze the last three layers
# for param in model.bert.encoder.layer[-2:].parameters():
#     param.requires_grad = True

for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True


##  Training

In [48]:
training_args = TrainingArguments(
    output_dir='my_best_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data['data'],
    eval_dataset=val_data['data'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.674048,0.584444,0.561743,0.974790,0.712750
2,No log,0.663286,0.586667,0.562500,0.983193,0.715596
3,No log,0.653779,0.626667,0.603550,0.857143,0.708333
4,No log,0.647448,0.655556,0.636964,0.810924,0.713494
5,No log,0.642972,0.642222,0.619938,0.836134,0.711986
6,No log,0.638813,0.668889,0.658363,0.777311,0.712909
7,No log,0.636090,0.671111,0.656250,0.794118,0.718631
8,0.660900,0.634811,0.662222,0.644295,0.806723,0.716418
9,0.660900,0.633790,0.664444,0.647458,0.802521,0.716698
10,0.660900,0.633190,0.675556,0.659722,0.798319,0.722433


TrainOutput(global_step=660, training_loss=0.6572014432964903, metrics={'train_runtime': 124.3775, 'train_samples_per_second': 168.841, 'train_steps_per_second': 5.306, 'total_flos': 755977990409616.0, 'train_loss': 0.6572014432964903, 'epoch': 10.0})

#### precision, recall, F1, accuracy  on Training set

In [53]:
trainer.evaluate(train_data['data'])

{'eval_loss': 0.6420993804931641,
 'eval_accuracy': 0.6271428571428571,
 'eval_precision': 0.6271439224459359,
 'eval_recall': 0.748220640569395,
 'eval_f1': 0.6823529411764705,
 'eval_runtime': 7.7612,
 'eval_samples_per_second': 270.577,
 'eval_steps_per_second': 8.504,
 'epoch': 10.0}

#### precision, recall, F1, accuracy  on validation set

In [54]:
trainer.evaluate(val_data['data'])

{'eval_loss': 0.6331900954246521,
 'eval_accuracy': 0.6755555555555556,
 'eval_precision': 0.6597222222222222,
 'eval_recall': 0.7983193277310925,
 'eval_f1': 0.7224334600760457,
 'eval_runtime': 1.4353,
 'eval_samples_per_second': 313.519,
 'eval_steps_per_second': 10.451,
 'epoch': 10.0}

#### precision, recall, F1, accuracy  on test set

In [55]:
trainer.evaluate(test_data['data'])

{'eval_loss': 0.6548696756362915,
 'eval_accuracy': 0.6022222222222222,
 'eval_precision': 0.6118881118881119,
 'eval_recall': 0.720164609053498,
 'eval_f1': 0.6616257088846881,
 'eval_runtime': 1.6653,
 'eval_samples_per_second': 270.223,
 'eval_steps_per_second': 9.007,
 'epoch': 10.0}